In [1]:
import ast
import numpy as np
%matplotlib inline
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('seaborn-whitegrid')
from sklearn import linear_model
plt.rcParams["figure.figsize"] = (16,8)
import math
import seaborn as sns
from sklearn.feature_extraction import DictVectorizer
import umap
import hdbscan
import sklearn.cluster as cluster
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score
import functions

In [2]:
import re
from random import choice

In [3]:
read = ''
for i in range(150): read += choice(['A', 'T', 'C', 'G'])
read

'CGTCGTCATTCTATTACATATAAAAATGCCACACCATTCTAAGGGTGGACTGTCAGTTTCCAGACTAAACCGGCCTTCCCTTCTACACCTATACTGCTGTATGATTGGCTCTTATTCTTCAGATATCTTGATTCGCATTAAATATAACAC'

In [4]:
def nuc_to_num(read):
    return read.replace('A', '1').replace('C', '2').replace('G', '3').replace('T', '4')

In [5]:
def convertdatatype(read):
    try: inverse = int(nuc_to_num(read[::-1]), 5)
    except: return -1
    verse = int(nuc_to_num(read), 5)
    if verse < inverse: return verse
    return inverse

In [6]:
def nuc_to_base32(read):
    return np.base_repr(convertdatatype(read), base=32)

In [7]:
def base32_to_nuc(val):
    dna_enc = np.base_repr(int(val, 32), base=5)
    return dna_enc.replace('1', 'A').replace('2', 'C').replace('3', 'G').replace('4', 'T')

In [8]:
def cut_to_kmer(read, length):
    return (re.findall('.{'+str(length)+'}', read))

def get_kmers(read, ksize):
    return [read[i:i+ksize] for i in range(0, len(read)-ksize+1)]

    

In [74]:
def get_kmers_from_read(read, kmer_len):
    kmers = np.array(list(map(nuc_to_base32, get_kmers(read, kmer_len))))
    return kmers[kmers != '-1']

In [75]:
def get_kmers_from_read_int(read, kmer_len):
    kmers = np.array(list(map(convertdatatype, get_kmers(read, kmer_len))))
    return kmers[kmers != '-1']

In [11]:
%timeit get_kmers_from_read(read, 40)


733 µs ± 23.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [15]:
get_kmers_from_read(read, 40)[:10]

array(['42Q7L8HOSIOABVRDCGO', '2Q76IL4QRF32QFTH60R',
       '2122BM06MTKOHQO7DF5', '42LK9OJNN5ORM9HBKLK',
       '5BRUNVJ18K82RTJIF8H', '5FDSFRKNFJU24PE2R1T',
       '3GM0CG95JESGU48HFOT', '2T7NEVLEJ054P9JFK6I',
       '5I6AI8DUIQ9MCBGP7M4', '2IL356L7EPFM26LLQJD'], dtype='<U19')

In [16]:
import os
import subprocess
#os.system('split -l 50000 example_100000_reads.txt')
int(subprocess.check_output('wc -l xaa.fastq', shell=True).split()[0])%4

0

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

### better formating to H5 instead of fastq

In [18]:
from pathlib import Path
import numpy as np
import os

In [25]:
def Fastq2KmersCompressed(filename, k = 30):
    with open(filename) as FQfile:
        with open(filename.split('.')[0] + '_kmers_b32.txt', 'w') as to_fil:
            while 1:
                if not FQfile.readline(): break
                to_fil.write(' '.join(get_kmers_from_read(FQfile.readline(),k)))
                FQfile.readline()
                FQfile.readline()
                
def fastq2kmerstxt(filename, k = 30):
    with open(filename) as FQfile:
        with open(filename.split('.')[0] + '_kmers.txt', 'w') as to_fil:
            while 1:
                if not FQfile.readline(): break
                to_fil.write(' '.join(get_kmers(FQfile.readline(), k)))
                FQfile.readline()
                FQfile.readline()
                
    
def Read_NPY_file(filename):
    stacks = np.array([])
    with open(filename,'rb') as f:
        for i in range(int(int(subprocess.check_output('wc -l ' + filename.split('_kmers')[0] + '.fastq', shell=True).split()[0])/4)):
            stacks = np.hstack([np.load(f, allow_pickle=True), stacks])
    return stacks

In [26]:
%%time
Fastq2KmersCompressed('xaa.fastq', 30)

CPU times: user 11.6 s, sys: 75.8 ms, total: 11.7 s
Wall time: 11.7 s


In [27]:
%%time 
fastq2kmerstxt('xaa.fastq', 30)

CPU times: user 391 ms, sys: 40.1 ms, total: 431 ms
Wall time: 432 ms


In [ ]:
%%time
#kmer_list = Read_NPY_file('xaa_kmers.npy')
#print(kmer_list.shape)


In [36]:
def fastq2kmersH5(filename, ksize = 30, enc = 'base64'):
    df = pd.HDFStore(filename.split('.')[0] + "_kmers.h5", 'w')
    if enc == 'base64':
        with open(filename) as FQfile:
            while 1:
                line = FQfile.readline()
                if not line: break
                df.append('seq',pd.DataFrame(get_kmers_from_read(FQfile.readline(), ksize), columns = ['kmers']), index=False )
                FQfile.readline()
                FQfile.readline()
    elif enc == 'string':
        with open(filename) as FQfile:
            while 1:
                line = FQfile.readline()
                if not line: break
                df.append('seq',pd.DataFrame(get_kmers(FQfile.readline(), ksize), columns = ['kmers']), index=False )
                FQfile.readline()
                FQfile.readline()
    df.close()
    
def fastq2tsv(filename):
    with open(filename) as FQfile:
        with open(filename.split('.')[0] + '.tsv', 'w') as to_fil:
            while 1:
                if not FQfile.readline(): break
                to_fil.write('\t'.join([FQfile.readline().replace('\n', ''), FQfile.readline().split(' ')[0][1:], FQfile.readline().replace('\n', '')]))
                to_fil.write('\n')


In [37]:
%%time
fastq2kmersH5('xaa.fastq', 30, enc = 'base64')


CPU times: user 40.5 s, sys: 571 ms, total: 41 s
Wall time: 41.3 s


In [38]:
%%time
fastq2tsv('example_100000_reads.txt')

CPU times: user 199 ms, sys: 52.2 ms, total: 251 ms
Wall time: 270 ms


# Open file Read, get Dict, make Sparse matrix

In [68]:
%%time
datah5  = pd.read_hdf('xaa_kmers.h5', key='seq')
datah5

CPU times: user 1.34 s, sys: 96.4 ms, total: 1.43 s
Wall time: 1.48 s


,kmers
0,IRI7V10CHCG0DC
1,NUUBLSCVK6LUD9
2,LUQQDNJ3Q7HU49
3,8R0HM8V1FGRV0A
4,H4V46UA5OMOK2S
...,...
183,GDD31Q2LCRAS3B
184,DC6C1PMVKH54P5
185,I2G40VD8SL7TOD
186,EL4EA39V8IHB29


In [40]:
%%time
data  = pd.read_csv('example_100000_reads.tsv', sep = '\t', header=None, names=['sequence', 'name', 'quality'])
data

CPU times: user 372 ms, sys: 52.3 ms, total: 424 ms
Wall time: 438 ms


,sequence,name,quality
0,GTTAGGGTTAGGGTTGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAG...,SRR925811.1,GGGGGGGGGGGGGEGGGGEGFFFFEFEFFFFGGGGFDFFFFFDFFE...
1,CCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTA...,SRR925811.2,GGDG?GGGEGG@G:GGGGGE?GBBGGGGGDGGEG@GGEB<CAAA>A...
2,TAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGG...,SRR925811.3,IIIIIIIIIIIIIIIIIIIGGGGGGEGGGFGGGGGIIGIHGFGGGI...
3,TAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGG...,SRR925811.4,GGGGGGGBGGGGGGGGGGGDGGGGGDGGGDB@EDDGGEGGBDGDGG...
4,CCTAACCCTAACCCCTAACCCCTAACCCTAACCCTAACCCTAACCC...,SRR925811.5,HHGHEHHHHHDHHGHHHHHHHHHHHHGHDHHHGHHDHHEHHHHHG@...
...,...,...,...
99995,CATGCCTCCTGTCCTGGGGGCAGGGAGGTGCTGCTGCCTGTATTTG...,SRR925811.99996,IIIIHIIIIIIIIHHIIIIGIHIIIIIIFIGI<IIIIGIIHIIIIG...
99996,CCAGTCCCAGCGTCCCCATTTGGCCCAGGCTGCTTCTCACCTTTGA...,SRR925811.99997,HHHHHHHHHHHGHHHHHGHHHHHHHHHHHHHHHHHHHHGHHDHHHH...
99997,GGCAGGGAGGTGCTGCTGCCTGTATTTGAAAGGAAGGGCATTGCGC...,SRR925811.99998,GGGGGGGGGGGGGGGEDEEGGGGGGGGGEGGGGFGGFGGAFAEFFG...
99998,AGGTAGTGTCCCCCGAACCTGTAGGCCTCGAAGGTGAGGGACAGGG...,SRR925811.99999,GEFAGDGEDGGGGGG27@@==<B<BFGEDFDEGGD?>DDDDBDGGE...


In [49]:
%%time
data['kmers'] = data.sequence.apply(get_kmers_from_read, kmer_len= 30)

CPU times: user 1min 30s, sys: 354 ms, total: 1min 30s
Wall time: 1min 29s


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

In [ ]:
corpus = []

# opening file with pyspark

In [65]:
from pyspark import SparkContext
from pyspark import SparkConf

# umap of kmers

In [72]:
import umap

In [73]:
data

,sequence,name,quality,kmers
0,GTTAGGGTTAGGGTTGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAG...,SRR925811.1,GGGGGGGGGGGGGEGGGGEGFFFFEFEFFFFGGGGFDFFFFFDFFE...,"[IRI7V10CHCG0DC, NUUBLSCVK6LUD9, LUQQDNJ3Q7HU4..."
1,CCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTA...,SRR925811.2,GGDG?GGGEGG@G:GGGGGE?GBBGGGGGDGGEG@GGEB<CAAA>A...,"[BFMI7OQ9IC53B8, CCL29CLA2E3VQO, CIEBSTDTC1MV4..."
2,TAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGG...,SRR925811.3,IIIIIIIIIIIIIIIIIIIGGGGGGEGGGFGGGGGIIGIHGFGGGI...,"[LUQQDNA6IM3EL2, 8R0HM7IF9PJHKA, A1FF7TQFGGAOC..."
3,TAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGG...,SRR925811.4,GGGGGGGBGGGGGGGGGGGDGGGGGDGGGDB@EDDGGEGGBDGDGG...,"[LUQQDNA6IM3EL2, 8R0HM7IF9PJHKA, A1FF7TQFGGAOC..."
4,CCTAACCCTAACCCCTAACCCCTAACCCTAACCCTAACCCTAACCC...,SRR925811.5,HHGHEHHHHHDHHGHHHHHHHHHHHHGHDHHHGHHDHHEHHHHHG@...,"[9ISTARRECLPI71, 6UU1VM73OA6J78, BFMI7OU8T7AM6..."
...,...,...,...,...
99995,CATGCCTCCTGTCCTGGGGGCAGGGAGGTGCTGCTGCCTGTATTTG...,SRR925811.99996,IIIIHIIIIIIIIHHIIIIGIHIIIIIIFIGI<IIIIGIIHIIIIG...,"[C1QAQHF4C54NIQ, 9PGSTGRD2V8Q3I, N0BKK8GA1TQJH..."
99996,CCAGTCCCAGCGTCCCCATTTGGCCCAGGCTGCTTCTCACCTTTGA...,SRR925811.99997,HHHHHHHHHHHGHHHHHGHHHHHHHHHHHHHHHHHHHHGHHDHHHH...,"[CF7PD5ODMIJMLD, BSK5QM9RN2JLGJ, 8VI3U911PIJFO..."
99997,GGCAGGGAGGTGCTGCTGCCTGTATTTGAAAGGAAGGGCATTGCGC...,SRR925811.99998,GGGGGGGGGGGGGGGEDEEGGGGGGGGGEGGGGFGGFGGAFAEFFG...,"[6RM0SNGIOA3K3V, 6DFS9LBB11F0D1, BRHTDCLRE0BS9..."
99998,AGGTAGTGTCCCCCGAACCTGTAGGCCTCGAAGGTGAGGGACAGGG...,SRR925811.99999,GEFAGDGEDGGGGGG27@@==<B<BFGEDFDEGGD?>DDDDBDGGE...,"[8RGE6QF35TBNGK, 8L7SRHV0DADCHC, 6P089DR0UR459..."


In [71]:
data.drop(['quality'])

KeyError: "['quality'] not found in axis"

In [79]:
import sys
read

'CGTCGTCATTCTATTACATATAAAAATGCCACACCATTCTAAGGGTGGACTGTCAGTTTCCAGACTAAACCGGCCTTCCCTTCTACACCTATACTGCTGTATGATTGGCTCTTATTCTTCAGATATCTTGATTCGCATTAAATATAACAC'

In [86]:
print(sys.getsizeof(get_kmers_from_read(read,10)))
print(sys.getsizeof(get_kmers_from_read_int(read,10)))


2916
1240


/home/thiagoms/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  This is separate from the ipykernel package so we can avoid doing imports until


In [80]:
get_kmers_from_read(read)

TypeError: get_kmers_from_read() missing 1 required positional argument: 'kmer_len'

In [ ]:
import sys

In [ ]:
read

In [ ]:
val = np.base_repr(int(nuc_to_num(read), 5), base=32)

In [ ]:
np.base_repr(int(val, 32), base=5) == nuc_to_num(read)

In [ ]:
np.base_repr(int(nuc_to_num(read), 5), base=5) == nuc_to_num(read)

In [51]:
np.base_repr(int(nuc_to_num(read), 5), base=32)

'5E28CN1VKGF04TL289AVRT4MGS3C1L8MV28J3N4C5322RPQJGAO1JGH6789R31PMG8O2T8'

In [64]:
read

'CGTCGTCATTCTATTACATATAAAAATGCCACACCATTCTAAGGGTGGACTGTCAGTTTCCAGACTAAACCGGCCTTCCCTTCTACACCTATACTGCTGTATGATTGGCTCTTATTCTTCAGATATCTTGATTCGCATTAAATATAACAC'

In [ ]:
np.base_repr(int(val, 32), base=5)

In [ ]:
nuc_to_base32('ATCGIDS')

In [61]:
int('5E28CN1VKGF04TL289AVRT4MGS3C1L8MV28J3N4C5322RPQJGAO1JGH6789R31PMG8O2T8', 32)

389873936670596444978611186223617578095584948919298065164920100653033808696041843851408934440261194091432

In [62]:
'E28CN1VKGF04TL289AVRT4MGS3C1L8MV28J3N4C5322RPQJGAO1JGH6789R31PMG8O2T87'[1:-1]

'28CN1VKGF04TL289AVRT4MGS3C1L8MV28J3N4C5322RPQJGAO1JGH6789R31PMG8O2T8'

In [ ]:
int('5E28CN1VKGF04TL289AVRT4MGS3C1L8MV28J3N4C5322RPQJGAO1JGH6789R31PMG8O2T8', 32)//10

In [ ]:
if st_1[1:30] == 